In [1]:
import numpy as np
import random
from data import Synthesize

PERIOD_LENGTH = 6 # numero de horas de cada periodo
N_PERIODS_YEAR = int((24/PERIOD_LENGTH) * 365) # numero de periodos per year

# conjuntos
sources = ['solar', 'wind', 'hydroelectric', 'nuclear', 'gas', 'coal', 'oil'] # conjunto _i
periods = [i for i in range(1, N_PERIODS_YEAR+1)] # conjunto _t

# parametros constantes
budget = 7000000000000 # presupuesto en dolares

years_return = 250 # cantidad de anos en los que se espera recuperar la inversion inicial

demand = Synthesize().synthesize_data(N_PERIODS_YEAR) # diccionario tipo {periodo:demanda}

battery_cost = 103240 # costo por bateria en dolares AC
battery_capacity = 232 # capacidad de almacenamiento en kWh por bateria AQ
battery_efficiency = 0.87 # eficiencia de energia de las baterias AE
pollution_per_battery = 46400 # kgs de CO2 emitidos por bateria producida AZ

kwh_price_mean = 0.1 # valor por kWh en el mercado promedio en dolares
kwh_price_chile = 0.16 # valor por kWh en chile en dolares

# parametro CI_i costo inicial por kW de capacidad en dolares
overnight_cost = {
    'solar': [1331, 1313, 1100],
    'wind': [1100, 1500, 1319],
    'hydroelectric': [5316, 1750, 7500, 3500],
    'nuclear': [6041, 6317, 6900, 12000],
    'gas': [950, 713, 1084, 1300],
    'coal': [3000, 6250, 5876, 3676],
    'oil': [1170, 1175, 713],
    }

# modelaremos ocupando la media en casos donde hayan mas de un dato
overnight_cost_mean = {
    key:np.mean(value) for key, value in  zip(overnight_cost.keys(), 
                                              overnight_cost.values())
}
    
# parametro C_i costo de produccion por kWh en dolares
production_cost = {
    'solar': [0.0017, 0.00136],
    'wind': [0.003, 0.0032],
    'hydroelectric': [0.005],
    'nuclear': [0.0210, 0.0138, 0.016],
    'gas': [0.021],
    'coal': [0.0098, 0.014, 0.0173],
    'oil': [0.054, 0.0701, 0.0563],
}

# al igual que el costo inicial, ocuparemos la media
production_cost_mean = {
    key:np.mean(value) for key, value in zip(production_cost.keys(),
                                            production_cost.values())
}

# kg de CO2 emitidos por producir un kWh Z_i
kg_CO2_per_kwh = {
    'solar': 0.085,
    'wind': 0.026,
    'hydroelectric': 0.026,
    'nuclear': 0.029,
    'gas': 0.499,
    'coal': 0.888,
    'oil': 0.733,
    }

# to-do: cantidad maxima disponible para instalaciones QM_i
prod_units_limit = {source:limit for source, limit in zip(sources, np.array([random.randint(10000000, 20000000) for _ in
                                                                            range(len(sources))]))}

defaultdict(<class 'int'>, {'1': 1867358417.9789155, '2': 1629281840.1285622, '3': 1439706902.0283077, '4': 1093207526.0548248, '5': 2083126733.1204906, '6': 1817541037.8008509, '7': 1606061218.134672, '8': 1219524757.7795396, '9': 2090703984.4580402, '10': 1824152236.7455728, '11': 1611903171.6364965, '12': 1223960707.5732508, '13': 2046257755.906563, '14': 1785372625.7486103, '15': 1577635758.697128, '16': 1197940554.6720114, '17': 1907631027.832493, '18': 1664419942.8394675, '19': 1470756514.0420756, '20': 1116784317.6133792, '21': 1860528517.146702, '22': 1623322709.150457, '23': 1434441145.1850398, '24': 1089209097.6223953, '25': 1985238530.0307858, '26': 1732132971.4535832, '27': 1530590799.4627554, '28': 1162218072.946389, '29': 1964752222.8670833, '30': 1714258540.9683836, '31': 1514796146.686575, '32': 1150224775.378657, '33': 2062627787.773078, '34': 1799655580.4217181, '35': 1590256821.4977837, '36': 1207524061.4406803, '37': 2192008458.3711042, '38': 1912541020.6454382, '39

In [2]:
from gurobipy import GRB, Model, quicksum

model = Model('Energy production planning')

prod_units = model.addVars(sources, vtype=GRB.INTEGER, name='prod_units')
storage = model.addVar(vtype=GRB.INTEGER, name='storage')
output = model.addVars(periods, sources, vtype=GRB.CONTINUOUS, name='output')
stored = model.addVars(periods, vtype=GRB.CONTINUOUS, name='stored')

model.update()

Using license file /home/rieg/gurobi.lic
Academic license - for non-commercial use only


In [3]:
initial_investment = quicksum(prod_units[source] * overnight_cost_mean[source] for source in sources)

# restriccion 1: inversion inicial menor o igual a presupuesto
model.addConstr(initial_investment <= budget)

# restriccion 2: recuperacion de inversion (el presupuesto dividido en la cantidad de anos nos indica
# cuanto debemos recuperar cada ano)
revenue = quicksum(output[period, source] * (kwh_price_mean - production_cost_mean[source]) \
                     for source in sources for period in periods)
# revenue es lo que se vende multiplicado por el precio menos el costo variable
model.addConstr(revenue - (storage * battery_cost / years_return) -\
                (initial_investment / years_return) >= (budget/years_return))

# restriccion 3: satisfaccion de demanda y flujo
model.addConstrs(stored[period] == quicksum(output[period, source] for source in sources) - demand[period] + \
                                            (stored[period - 1] * battery_efficiency) \
                                                for idx, period in enumerate(periods) if idx != 0)

# restriccion 4: produccion de energia limitada
model.addConstrs(output[period, source] <= prod_units[source] * PERIOD_LENGTH for source in sources
                                                                                for period in periods)
# restriccion 5: cantidad de instalaciones limitadas 
model.addConstrs(prod_units[source] <= prod_units_limit[source] for source in sources)

# restriccion 6: cantidad maxima de almacenamiento
model.addConstrs(stored[period] <= storage * battery_capacity for period in periods)

# Naturaleza de las variables
# restriccion 7:
model.addConstrs(output[period, source] >= 0 for source in sources for period in periods)
# restriccion 8:
model.addConstrs(prod_units[source] >= 0 for source in sources)
# restriccion 9:
model.addConstr(storage >= 0)
# restriccion 10 y 11:
model.addConstrs(stored[period] >= 0 for idx, period in enumerate(periods) if idx != 0)
model.addConstrs(stored[period] == 0 for idx, period in enumerate(periods) if idx == 0)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>}

In [4]:
obj = quicksum(output[period, source] * kg_CO2_per_kwh[source]\
               for source in sources for period in periods) + storage * pollution_per_battery

In [5]:
model.setObjective(obj, GRB.MINIMIZE)

model.optimize()

model.printAttr('X')
print('\n---------------------\n')
for constr in model.getConstrs():
    print(constr, constr.getAttr('slack'))
    
model.write('model.sol')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 24836 rows, 11688 columns and 58421 nonzeros
Model fingerprint: 0x749bb2fa
Variable types: 11680 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 8e+03]
  Objective range  [3e-02, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+07, 7e+12]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 11698 rows and 2 columns
Presolve time: 0.04s
Presolved: 13138 rows, 11686 columns, 46714 nonzeros
Variable types: 11678 continuous, 8 integer (0 binary)

Root relaxation: objective 3.560268e+11, 6811 iterations, 1.38 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.5603e+11    0    2          - 3.5603e+11      -     -    1s
H    0     0                    9.374889e+11 3.5603e+

output[66,solar]  5.66409e+07 
output[66,wind]  8.13663e+07 
output[66,hydroelectric]  8.47277e+07 
output[67,solar]  5.66409e+07 
output[67,wind]  8.13663e+07 
output[67,hydroelectric]  8.47277e+07 
output[68,solar]  5.66409e+07 
output[68,wind]  8.13663e+07 
output[68,hydroelectric]  8.47277e+07 
output[69,solar]  5.66409e+07 
output[69,wind]  8.13663e+07 
output[69,hydroelectric]  8.47277e+07 
output[70,solar]  5.66409e+07 
output[70,wind]  8.13663e+07 
output[70,hydroelectric]  8.47277e+07 
output[71,solar]  5.66409e+07 
output[71,wind]  8.13663e+07 
output[71,hydroelectric]  8.47277e+07 
output[72,solar]  5.66409e+07 
output[72,wind]  8.13663e+07 
output[72,hydroelectric]  8.47277e+07 
output[73,solar]  5.66409e+07 
output[73,wind]  8.13663e+07 
output[73,hydroelectric]  8.47277e+07 
output[74,solar]  5.66409e+07 
output[74,wind]  8.13663e+07 
output[74,hydroelectric]  8.47277e+07 
output[75,solar]  5.66409e+07 
output[75,wind]  8.13663e+07 
output[75,hydroelectric]  8.47277e+07 


output[146,hydroelectric]  8.47277e+07 
output[147,solar]  5.66409e+07 
output[147,wind]  8.13663e+07 
output[147,hydroelectric]  8.47277e+07 
output[148,solar]  5.66409e+07 
output[148,wind]  8.13663e+07 
output[148,hydroelectric]  8.47277e+07 
output[149,solar]  5.66409e+07 
output[149,wind]  8.13663e+07 
output[149,hydroelectric]  8.47277e+07 
output[150,solar]  5.66409e+07 
output[150,wind]  8.13663e+07 
output[150,hydroelectric]  8.47277e+07 
output[151,solar]  5.66409e+07 
output[151,wind]  8.13663e+07 
output[151,hydroelectric]  8.47277e+07 
output[152,solar]  5.66409e+07 
output[152,wind]  8.13663e+07 
output[152,hydroelectric]  8.47277e+07 
output[153,solar]  5.66409e+07 
output[153,wind]  8.13663e+07 
output[153,hydroelectric]  8.47277e+07 
output[154,solar]  5.66409e+07 
output[154,wind]  8.13663e+07 
output[154,hydroelectric]  8.47277e+07 
output[155,solar]  5.66409e+07 
output[155,wind]  8.13663e+07 
output[155,hydroelectric]  8.47277e+07 
output[156,solar]  5.66409e+07 
o

output[226,wind]  8.13663e+07 
output[226,hydroelectric]  8.47277e+07 
output[227,solar]  5.66409e+07 
output[227,wind]  8.13663e+07 
output[227,hydroelectric]  8.47277e+07 
output[228,solar]  5.66409e+07 
output[228,wind]  8.13663e+07 
output[228,hydroelectric]  8.47277e+07 
output[229,solar]  5.66409e+07 
output[229,wind]  8.13663e+07 
output[229,hydroelectric]  8.47277e+07 
output[230,solar]  5.66409e+07 
output[230,wind]  8.13663e+07 
output[230,hydroelectric]  8.47277e+07 
output[231,solar]  5.66409e+07 
output[231,wind]  8.13663e+07 
output[231,hydroelectric]  8.47277e+07 
output[232,solar]  5.66409e+07 
output[232,wind]  8.13663e+07 
output[232,hydroelectric]  8.47277e+07 
output[233,solar]  5.66409e+07 
output[233,wind]  8.13663e+07 
output[233,hydroelectric]  8.47277e+07 
output[234,solar]  5.66409e+07 
output[234,wind]  8.13663e+07 
output[234,hydroelectric]  8.47277e+07 
output[235,solar]  5.66409e+07 
output[235,wind]  8.13663e+07 
output[235,hydroelectric]  8.47277e+07 
ou

output[306,solar]  5.66409e+07 
output[306,wind]  8.13663e+07 
output[306,hydroelectric]  8.47277e+07 
output[307,solar]  5.66409e+07 
output[307,wind]  8.13663e+07 
output[307,hydroelectric]  8.47277e+07 
output[308,solar]  5.66409e+07 
output[308,wind]  8.13663e+07 
output[308,hydroelectric]  8.47277e+07 
output[309,solar]  5.66409e+07 
output[309,wind]  8.13663e+07 
output[309,hydroelectric]  8.47277e+07 
output[310,solar]  5.66409e+07 
output[310,wind]  8.13663e+07 
output[310,hydroelectric]  8.47277e+07 
output[311,solar]  5.66409e+07 
output[311,wind]  8.13663e+07 
output[311,hydroelectric]  8.47277e+07 
output[312,solar]  5.66409e+07 
output[312,wind]  8.13663e+07 
output[312,hydroelectric]  8.47277e+07 
output[313,solar]  5.66409e+07 
output[313,wind]  8.13663e+07 
output[313,hydroelectric]  8.47277e+07 
output[314,solar]  5.66409e+07 
output[314,wind]  8.13663e+07 
output[314,hydroelectric]  8.47277e+07 
output[315,solar]  5.66409e+07 
output[315,wind]  8.13663e+07 
output[315

...

<gurobi.Constr R23545> -1713345288.0
<gurobi.Constr R23546> -1713345288.0
<gurobi.Constr R23547> -1713345288.0
<gurobi.Constr R23548> -1713345288.0
<gurobi.Constr R23549> -1713345288.0
<gurobi.Constr R23550> -1713345288.0
<gurobi.Constr R23551> -1713345288.0
<gurobi.Constr R23552> -1713345288.0
<gurobi.Constr R23553> -1713345288.0
<gurobi.Constr R23554> -1713345288.0
<gurobi.Constr R23555> -1713345288.0
<gurobi.Constr R23556> -1713345288.0
<gurobi.Constr R23557> -1713345288.0
<gurobi.Constr R23558> -1713345288.0
<gurobi.Constr R23559> -1713345288.0
<gurobi.Constr R23560> -1713345288.0
<gurobi.Constr R23561> -1713345288.0
<gurobi.Constr R23562> -1713345288.0
<gurobi.Constr R23563> -1713345288.0
<gurobi.Constr R23564> -1713345288.0
<gurobi.Constr R23565> -1713345288.0
<gurobi.Constr R23566> -1713345288.0
<gurobi.Constr R23567> -1713345288.0
<gurobi.Constr R23568> -1713345288.0
<gurobi.Constr R23569> -1713345288.0
<gurobi.Constr R23570> -1713345288.0
<gurobi.Constr R23571> -1713345288.0
<

<gurobi.Constr R23903> -1713345288.0
<gurobi.Constr R23904> -1713345288.0
<gurobi.Constr R23905> -1713345288.0
<gurobi.Constr R23906> -1713345288.0
<gurobi.Constr R23907> -1713345288.0
<gurobi.Constr R23908> -1713345288.0
<gurobi.Constr R23909> -1713345288.0
<gurobi.Constr R23910> -1713345288.0
<gurobi.Constr R23911> -1713345288.0
<gurobi.Constr R23912> -1713345288.0
<gurobi.Constr R23913> -1713345288.0
<gurobi.Constr R23914> -1713345288.0
<gurobi.Constr R23915> -1713345288.0
<gurobi.Constr R23916> -1713345288.0
<gurobi.Constr R23917> -1713345288.0
<gurobi.Constr R23918> -1713345288.0
<gurobi.Constr R23919> -1713345288.0
<gurobi.Constr R23920> -1713345288.0
<gurobi.Constr R23921> -1713345288.0
<gurobi.Constr R23922> -1713345288.0
<gurobi.Constr R23923> -1713345288.0
<gurobi.Constr R23924> -1713345288.0
<gurobi.Constr R23925> -1713345288.0
<gurobi.Constr R23926> -1713345288.0
<gurobi.Constr R23927> -1713345288.0
<gurobi.Constr R23928> -1713345288.0
<gurobi.Constr R23929> -1713345288.0
<

<gurobi.Constr R24224> -1713345288.0
<gurobi.Constr R24225> -1713345288.0
<gurobi.Constr R24226> -1713345288.0
<gurobi.Constr R24227> -1713345288.0
<gurobi.Constr R24228> -1713345288.0
<gurobi.Constr R24229> -1713345288.0
<gurobi.Constr R24230> -1713345288.0
<gurobi.Constr R24231> -1713345288.0
<gurobi.Constr R24232> -1713345288.0
<gurobi.Constr R24233> -1713345288.0
<gurobi.Constr R24234> -1713345288.0
<gurobi.Constr R24235> -1713345288.0
<gurobi.Constr R24236> -1713345288.0
<gurobi.Constr R24237> -1713345288.0
<gurobi.Constr R24238> -1713345288.0
<gurobi.Constr R24239> -1713345288.0
<gurobi.Constr R24240> -1713345288.0
<gurobi.Constr R24241> -1713345288.0
<gurobi.Constr R24242> -1713345288.0
<gurobi.Constr R24243> -1713345288.0
<gurobi.Constr R24244> -1713345288.0
<gurobi.Constr R24245> -1713345288.0
<gurobi.Constr R24246> -1713345288.0
<gurobi.Constr R24247> -1713345288.0
<gurobi.Constr R24248> -1713345288.0
<gurobi.Constr R24249> -1713345288.0
<gurobi.Constr R24250> -1713345288.0
<

<gurobi.Constr R24546> -1713345288.0
<gurobi.Constr R24547> -1713345288.0
<gurobi.Constr R24548> -1713345288.0
<gurobi.Constr R24549> -1713345288.0
<gurobi.Constr R24550> -1713345288.0
<gurobi.Constr R24551> -1713345288.0
<gurobi.Constr R24552> -1713345288.0
<gurobi.Constr R24553> -1713345288.0
<gurobi.Constr R24554> -1713345288.0
<gurobi.Constr R24555> -1713345288.0
<gurobi.Constr R24556> -1713345288.0
<gurobi.Constr R24557> -1713345288.0
<gurobi.Constr R24558> -1713345288.0
<gurobi.Constr R24559> -1713345288.0
<gurobi.Constr R24560> -1713345288.0
<gurobi.Constr R24561> -1713345288.0
<gurobi.Constr R24562> -1713345288.0
<gurobi.Constr R24563> -1713345288.0
<gurobi.Constr R24564> -1713345288.0
<gurobi.Constr R24565> -1713345288.0
<gurobi.Constr R24566> -1713345288.0
<gurobi.Constr R24567> -1713345288.0
<gurobi.Constr R24568> -1713345288.0
<gurobi.Constr R24569> -1713345288.0
<gurobi.Constr R24570> -1713345288.0
<gurobi.Constr R24571> -1713345288.0
<gurobi.Constr R24572> -1713345288.0
<

<gurobi.Constr R24834> -1713345288.0
<gurobi.Constr R24835> 0.0


In [6]:
if model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")
    model.feasRelaxS(1, False, False, True)
    model.optimize()